#### Entrega - Modelo e Previsão

> Explique como você faria a previsão da nota do imdb a partir dos dados. Quais variáveis e/ou suas transformações você utilizou e por quê? Qual tipo de problema estamos resolvendo (regressão, classificação)? Qual modelo melhor se aproxima dos dados e quais seus prós e contras? Qual medida de performance do modelo foi escolhida e por quê?

Primeiramente, esse é um problema de regressão, estamos prevendo o valor de uma variável numérica (IMDB_Rating) a partir das variáveis independentes. Considerando que o dataset tem valores limitados de IMDB_Rating, por causa de algum filtro ou da origem do dataset, esse poderia também ser um problema de classificação ao criar categorías de baixo, médio ou alto rating IMDB, mas irei continuar como um problema de regressão.

Para fazer a previsão irei utilizar os dados e aplicar transformações e feature engineering para obter o máximo de informação possível dos dados, por exemplo:
- A partir do Overview pude utilizar um processo de NLP (NLTK + CountVectorizer) para transformar o em embeddings e conseguir as 300 keywords e com isso ter uma nova feature de No_keywords no dataset.
- Com Overview também criei a feature Overview_length, que é a quantidade total de palavras do Overview.
- Transformei Released_Year -> Released_Year_Group, pegando a divisão inteira por 10 e multiplicando por 10 para agrupar o ano de lançamento em agrupamentos de 10 em 10 anos.
- Separei o Runtime para Runtime_time e Runtime_category, como só temos filmes com o Runtime em minutos, posso descartar essa feature, com Runtime_time obtive o Runtime_category, com filmes short, medium ou long duration.
- Com genre irei aplicar uma transformação a depender do algoritmo que estiver usando, como Count ou Target Encoding, mas também criei uma feature No_Genres que conta o número de genêros no filme.
- Director também irei precisar aplicar uma transformação, como o Label Encodding por exemplo, mas também utilizei para criar a feature Director_No_Movies que conta quantos filmes o diretor produziu.

Em relação as outras variáveis númericas irei testar transformações como normalização e escalonamento.

Sobre os modelos e métricas irei descorrer melhor sobre isso no relatório completo (PDF)


In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MultiLabelBinarizer, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.impute import SimpleImputer


dataset =  pd.read_csv('./desafio_indicium_imdb_after_process.csv', index_col=0, sep=';')
dataset.head(3)
dataset.isna().sum()

Series_Title             0
Released_Year            0
Certificate              0
Runtime                  0
Genre                    0
IMDB_Rating              0
Overview                 0
Meta_score             157
Director                 0
Star1                    0
Star2                    0
Star3                    0
Star4                    0
No_of_Votes              0
Gross                    0
Runtime_time             0
Runtime_type             0
No_keywords              0
Released_Year_Group      0
Runtime_category         0
No_Genres                0
Director_No_Movies       0
Overview_length          0
dtype: int64

In [3]:
string_var = ['Series_Title', 'Overview']
cat_var = ['Certificate', 'Genre', 'Director', 'Star1', 'Star2', 'Star3', 'Star4']
numeric_var = ['Released_Year', 'Runtime_time', 'Meta_score', 'No_of_Votes','Gross', 'No_keywords', 'Released_Year_Group', 'No_Genres', 'Director_No_Movies', 'Overview_length']

X = dataset[string_var + cat_var + numeric_var]
y = dataset['IMDB_Rating']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
# Basemodel com Lasso Regression (L1)

from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split

num_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

genre_pipeline = Pipeline(
    steps=[
        ('spliter', FunctionTransformer(lambda x: x['Genre'].str.split(", "), validate=False)),
        ('multilabel_binarizer', MultiLabelBinarizer())
    ]
)
actor_pipeline = Pipeline(
    steps=[
        ('join', FunctionTransformer(lambda x: X[['Star1', 'Star2', 'Star3', 'Star4']].values.tolist(), validate=False)),
        ('multilabel_binarizer', MultiLabelBinarizer())
    ]
)
preprocessor = ColumnTransformer(transformers=[
    ('genres', genre_pipeline, [cat_var[1]]),
    ('stars', actor_pipeline, cat_var[3:]),
    ('cat_var', LabelEncoder(), [cat_var[0],cat_var[2]]),
    ('num_var', num_pipeline, numeric_var)
])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lasso', LassoCV(max_iter=100, cv=5, n_jobs=-1, random_state=42))
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)


KeyError: 'Genre'